In [363]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sample_submission.csv
/kaggle/input/test.csv
/kaggle/input/data_description.txt
/kaggle/input/train.csv


In [364]:
train_set = pd.read_csv("/kaggle/input/train.csv")
test_set = pd.read_csv("/kaggle/input/test.csv")
test_y1 = pd.read_csv("/kaggle/input/sample_submission.csv")
test_y = test_y1["SalePrice"]
print(train_set.shape)
print(test_set.shape)

(1460, 81)
(1459, 80)


In [365]:
train_set[['SaleType']].shape
train_set['SaleType']
XX = train_set.select_dtypes(include=[object])
categorical_feature_mask = train_set.dtypes==object
categorical_feature_mask = train_set.columns[categorical_feature_mask].tolist();
categorical_feature_mask

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [366]:
train_set['SalePrice']
corr_matrix =train_set.corr()
corr_matrix['SalePrice'].sort_values(ascending=False)


SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [367]:

needed_columns = ["OverallQual",'GrLivArea','GarageCars','GarageArea','TotalBsmtSF','1stFlrSF','FullBath','TotRmsAbvGrd','YearBuilt','YearRemodAdd',
            'GarageYrBlt','MasVnrArea','Fireplaces','BsmtFinSF1','LotFrontage','WoodDeckSF','2ndFlrSF','OpenPorchSF','HalfBath','LotArea','BsmtFullBath','BsmtUnfSF','BedroomAbvGr','ScreenPorch']
X = train_set[needed_columns]
y = train_set['SalePrice']
X.head()

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,...,LotFrontage,WoodDeckSF,2ndFlrSF,OpenPorchSF,HalfBath,LotArea,BsmtFullBath,BsmtUnfSF,BedroomAbvGr,ScreenPorch
0,7,1710,2,548,856,856,2,8,2003,2003,...,65.0,0,854,61,1,8450,1,150,3,0
1,6,1262,2,460,1262,1262,2,6,1976,1976,...,80.0,298,0,0,0,9600,0,284,3,0
2,7,1786,2,608,920,920,2,6,2001,2002,...,68.0,0,866,42,1,11250,1,434,3,0
3,7,1717,3,642,756,961,1,7,1915,1970,...,60.0,0,756,35,0,9550,1,540,3,0
4,8,2198,3,836,1145,1145,2,9,2000,2000,...,84.0,192,1053,84,1,14260,1,490,4,0


In [368]:
null_columns=X.columns[X.isnull().any()]
X[null_columns].isnull().sum()

GarageYrBlt     81
MasVnrArea       8
LotFrontage    259
dtype: int64

In [369]:
median = int(X['MasVnrArea'].median())
X['MasVnrArea'].fillna(int(median),inplace=True)
median = X['LotFrontage'].median()
X['LotFrontage'].fillna(int(median),inplace=True)
median = X['GarageYrBlt'].median()
X['GarageYrBlt'].fillna(int(median),inplace=True)
print(X.shape)
print(X.shape)
test_X  = test_set[needed_columns]
print(test_X.shape)
null_columns=X.columns[X.isnull().any()]
X[null_columns].isnull().sum()
null_columns=test_X.columns[test_X.isnull().any()]
test_X[null_columns].isnull().sum()

(1460, 24)
(1460, 24)
(1459, 24)


/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


GarageCars        1
GarageArea        1
TotalBsmtSF       1
GarageYrBlt      78
MasVnrArea       15
BsmtFinSF1        1
LotFrontage     227
BsmtFullBath      2
BsmtUnfSF         1
dtype: int64

In [370]:
median = test_X['MasVnrArea'].median()
test_X['MasVnrArea'].fillna(int(median),inplace=True)
median = test_X['BsmtFinSF1'].median()
test_X['BsmtFinSF1'].fillna(median,inplace=True)
median = test_X['LotFrontage'].median()
test_X['LotFrontage'].fillna(int(median),inplace=True)
median = test_X['BsmtFullBath'].median()
test_X['BsmtFullBath'].fillna(median,inplace=True)
median = test_X['BsmtUnfSF'].median()
test_X['BsmtUnfSF'].fillna(median,inplace=True)
median = test_X['GarageYrBlt'].median()
test_X['GarageYrBlt'].fillna(int(median),inplace=True)
median = test_X['GarageCars'].median()
test_X['GarageCars'].fillna(median,inplace=True)
median = test_X['GarageArea'].median()
test_X['GarageArea'].fillna(median,inplace=True)
median = test_X['TotalBsmtSF'].median()
test_X['TotalBsmtSF'].fillna(median,inplace=True)
null_columns=test_X.columns[test_X.isnull().any()]
test_X[null_columns].isnull().sum()

Series([], dtype: float64)

In [371]:
null_columns=X.columns[X.isnull().any()]
X[null_columns].isnull().sum()
null_columns=test_X.columns[test_X.isnull().any()]
test_X[null_columns].isnull().sum()

Series([], dtype: float64)

In [372]:
from sklearn.preprocessing import MinMaxScaler

In [373]:

scaler = MinMaxScaler()
X_t = pd.DataFrame(scaler.fit_transform(X));
X_te = pd.DataFrame(scaler.fit_transform(test_X));
# print(X[:5])
# print(X_t[:5])


max= train_set['SalePrice'].max()
y = y/max
test_y = test_y/max


In [374]:
from tensorflow import keras
from keras.models import Sequential
# model  = keras.Sequential()
# model
# model.add(keras.layers.Input(shape=X.shape[1:]));
# model.add(keras.layers.Dense(20, activation="relu"));
# model.add(keras.layers.Dense(10, activation="relu"));
# model.add(keras.layers.Dense(1));
# input1 = keras.layers.Input(shape=X.shape[1:])
# hidden1 = keras.layers.Dense(300, activation="relu")(input1)
# hidden2 = keras.layers.Dense(300, activation="relu")(hidden1)
# hidden3  = keras.layers.Dense(100, activation="relu")(hidden2)
# output = keras.layers.Dense(1)(hidden3)
from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense
three_layer_model = Sequential()
three_layer_model.add(Dense(100, activation='relu',input_dim=X.shape[1:][0]))
three_layer_model.add(Dense(50,activation='relu'))
three_layer_model.add(Dense(output_dim=1,activation='linear'))
three_layer_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])




/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`


In [375]:
history=three_layer_model.fit(X_t,y,epochs=30)
predictions = three_layer_model.predict(X_te)
print(predictions[:5],"  ", test_y[:5])
#history.history

Epoch 1/30
1460/1460 [==============================] - 1s 907us/step - loss: 0.0058 - acc: 6.8493e-04
Epoch 2/30
1460/1460 [==============================] - 0s 65us/step - loss: 0.0026 - acc: 6.8493e-04
Epoch 3/30
1460/1460 [==============================] - 0s 64us/step - loss: 0.0023 - acc: 6.8493e-04
Epoch 4/30
1460/1460 [==============================] - 0s 60us/step - loss: 0.0021 - acc: 6.8493e-04
Epoch 5/30
1460/1460 [==============================] - 0s 60us/step - loss: 0.0019 - acc: 6.8493e-04
Epoch 6/30
1460/1460 [==============================] - 0s 59us/step - loss: 0.0017 - acc: 6.8493e-04
Epoch 7/30
1460/1460 [==============================] - 0s 54us/step - loss: 0.0018 - acc: 6.8493e-04
Epoch 8/30
1460/1460 [==============================] - 0s 58us/step - loss: 0.0018 - acc: 6.8493e-04
Epoch 9/30
1460/1460 [==============================] - 0s 53us/step - loss: 0.0015 - acc: 6.8493e-04
Epoch 10/30
1460/1460 [==============================] - 0s 54us/step - loss: 0.0

In [376]:
from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense
five_layer_model = Sequential()
five_layer_model.add(Dense(300, activation='relu',input_dim=X.shape[1:][0]))
five_layer_model.add(Dense(100,activation='relu'))
five_layer_model.add(Dense(50,activation='relu'))
five_layer_model.add(Dense(10,activation='relu'))
five_layer_model.add(Dense(output_dim=1,activation='linear'))
five_layer_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  if __name__ == '__main__':


In [377]:
history=five_layer_model.fit(X_t,y,epochs=30)
predictions = five_layer_model.predict(X_te)
print(predictions[:5],"  ", test_y[:5])

Epoch 1/30
1460/1460 [==============================] - 2s 1ms/step - loss: 0.0069 - acc: 0.0000e+00
Epoch 2/30
1460/1460 [==============================] - 0s 70us/step - loss: 0.0025 - acc: 6.8493e-04
Epoch 3/30
1460/1460 [==============================] - 0s 64us/step - loss: 0.0021 - acc: 6.8493e-04
Epoch 4/30
1460/1460 [==============================] - 0s 68us/step - loss: 0.0020 - acc: 6.8493e-04
Epoch 5/30
1460/1460 [==============================] - 0s 72us/step - loss: 0.0018 - acc: 6.8493e-04
Epoch 6/30
1460/1460 [==============================] - 0s 69us/step - loss: 0.0016 - acc: 6.8493e-04
Epoch 7/30
1460/1460 [==============================] - 0s 67us/step - loss: 0.0016 - acc: 6.8493e-04
Epoch 8/30
1460/1460 [==============================] - 0s 65us/step - loss: 0.0015 - acc: 6.8493e-04
Epoch 9/30
1460/1460 [==============================] - 0s 66us/step - loss: 0.0015 - acc: 6.8493e-04
Epoch 10/30
1460/1460 [==============================] - 0s 65us/step - loss: 0.001

In [378]:
print(predictions[:5]*max,"  ", test_y[:5]*max)

[[141680.29144406]
 [165169.52544451]
 [210101.97743773]
 [215023.22971821]
 [193596.97729349]]    0    169277.052498
1    187758.393989
2    183583.683570
3    179317.477511
4    150730.079977
Name: SalePrice, dtype: float64


In [379]:
submission = pd.read_csv('/kaggle/input/test.csv')
submit = submission[['Id']].copy()
predictions = predictions * max;
submit['SalePrice'] = predictions
submit.to_csv('submission.csv',index=False)